<a href="https://colab.research.google.com/github/ffelfis/OrgaDatosTPs/blob/main/TP2/resources/OPT_hyperopt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lectura de datos de Google Drive

In [ ]:
# Lectura de Dataset desde Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Para importar funciones customizadas

Hay que especificar la ruta de donde se encuentra el módulo (archivo `.py`) para buscar las funciones.

La lectura puede ser muy celosa: las indentaciones son de 4 espacios no tabulaciones.

https://colab.research.google.com/drive/1uvHuizCBqFgvbCwEhK7FvU8JW0AfxgJw

In [ ]:
import sys
sys.path.append('/content/drive/My Drive/75.06 - Organización de Datos/TP2/resources')

# Carga de librerías y directorios

In [ ]:
import pandas as pd
import numpy as np

# Métrica de evaluación
from sklearn.metrics import f1_score
# fold_score = f1_score(y_test, prediction, average='micro')
# Se especifica average por tener un target multiclase

# Clasificador
from sklearn.ensemble import RandomForestClassifier

# Para la división en k-folds
from sklearn.model_selection import StratifiedKFold

# Para pasar parámetros varias veces a una función
from functools import partial

# Paquetes de hyperopt
from hyperopt import hp, fmin, tpe, Trials
# Cuando se especifica un espacio de distribuciones de Integers a veces
# devuelve Floats. scope resuelve este problema
from hyperopt.pyll.base import scope

# Librería para guardar los Trials.
import pickle

# Función para cambiar tipos de datos
from utilidades import cambio_tipos

# Rutas de los archivos a usar

In [ ]:
# Ruta train_values.csv
dir_values = '/content/drive/My Drive/75.06 - Organización de Datos/TP1/Data/train_values.csv'
# Ruta train_labels.csv
dir_labels = '/content/drive/My Drive/75.06 - Organización de Datos/TP1/Data/train_labels.csv'
# Ruta de Binary Encodings para train_values.csv
dir_resources = '/content/drive/My Drive/75.06 - Organización de Datos/TP2/resources'

---
#Entrenamiento
---
### Carga de train

In [ ]:
columnas = ['building_id',
 'geo_level_1_id',
 'geo_level_2_id',
 'geo_level_3_id',
 'count_floors_pre_eq',
 'age',
 'area_percentage',
 'height_percentage',
 'has_superstructure_adobe_mud',
 'has_superstructure_mud_mortar_stone',
 'has_superstructure_stone_flag',
 'has_superstructure_cement_mortar_stone',
 'has_superstructure_mud_mortar_brick',
 'has_superstructure_cement_mortar_brick',
 'has_superstructure_timber',
 'has_superstructure_bamboo',
 'has_superstructure_rc_non_engineered',
 'has_superstructure_rc_engineered',
 'has_superstructure_other',
 'count_families',
 'has_secondary_use',
 'has_secondary_use_agriculture',
 'has_secondary_use_hotel',
 'has_secondary_use_rental',
 'has_secondary_use_institution',
 'has_secondary_use_school',
 'has_secondary_use_industry',
 'has_secondary_use_health_post',
 'has_secondary_use_gov_office',
 'has_secondary_use_use_police',
 'has_secondary_use_other']
 
# Carga de train_values.csv
train = pd.read_csv(dir_values, usecols=columnas)

### Cambio de tipos de datos

In [ ]:
train = cambio_tipos(train)

### Carga de columnas codificadas: Binary Encoding

In [ ]:
# 28 columnas más.
train = train.join(pd.read_csv(dir_resources+f'/BE_train.csv', dtype='uint8'))

X = train.values

### Carga de labels

In [ ]:
# Carga de train_labels.csv
labels = pd.read_csv(dir_labels, usecols=['damage_grade'], dtype='uint8')

y = labels.damage_grade.values

# Hyperopt
Usa Tree-Structured Parzen Estimator (TPE)

- Aquí la función de optimización recibe directamente un diccionario.
- Aquí se define la cantidad de __splits para Cross Validation__

In [ ]:
# params: lista de valores de parámetros (value)
# x: features
# y: target

def optimize(params, x, y):
  # Se instancia el modelo con el diccionario de parámetros
  model = RandomForestClassifier(**params)
  # Hay que realizar el k-folding
  kf = StratifiedKFold(n_splits=5)
  # Lista de scores
  scores = []
  # Se dividen los datos
  for index in kf.split(X=x, y=y):
    train_index, test_index = index[0], index[1]
    X_train = x[train_index]
    y_train = y[train_index]
    X_test = x[test_index]
    y_test = y[test_index]
    # Entrenamiento del k-fold
    model.fit(X_train, y_train)
    prediction = model.predict(X_test)
    # average='micro' es necesario por ser un target multiclase
    fold_score = f1_score(y_test, prediction, average='micro')
    # Se agrega el score a la lista
    scores.append(fold_score)
  # Ahora hay que devolver la función para minimizar
  return -1.0 * np.mean(scores)

Para definir el espacio de parámetros hay que usar un diccionario y usar espacios dedicados de `hyperopt`.

In [ ]:
# Los valores que requieren enteros tienen un scope de enteros
param_space = {
  'max_depth' : scope.int(hp.quniform('max_depth', 40, 60, 1)),
  'n_estimators' : scope.int(hp.quniform('n_estimators', 10, 50, 1)),
  'criterion' : hp.choice('criterion', ['gini', 'entropy']),
  'max_features' : hp.uniform('max_features', 0.05, 1)
}

Función para optimizar, sin `param_names`.

In [ ]:
optimization_function = partial(optimize, x=X, y=y)

Se inicializan los trials. Para continuar con la optimización conviene tener por separado la celda que busca minimizar la función objetivo. Así los Trials acarrean las combinaciones usadas anteriormente.

In [ ]:
trials = Trials()

Aquí se usa la función `fmin` para minimizar.

In [ ]:
result = fmin(fn=optimization_function, space=param_space, algo=tpe.suggest, max_evals=5, trials=trials)

100%|██████████| 5/5 [21:42<00:00, 260.56s/it, best loss: -0.729083917915015]


In [35]:
print(result)

{'criterion': 1, 'max_depth': 48.0, 'max_features': 0.8310276775795518, 'n_estimators': 47.0}


`criterion` indica la pocisión en el vector del parámetro indicado.

### Almacenamiento de Trials
Para continuar la optimización.

_Observación_: Recordar que luego hay que aumentar el número de `max_evals`, en `fmin`, para seguir acumulando pruebas.

In [ ]:
# String con dirección de almacenamiento y nombre del archivo .pkl.
trials_file_name = 'pickle/Trials_LGBMClassifier05.pkl'

# Se crea una variable pickle y se abre en modo de escritura.
# [También se crea el archivo en el directorio especificado por lgbm_file_name]
trials_pickle = open(trials_file_name, 'wb')

# Se usa el método dump con el modelo y el archivo creado en modo de esritura
# para crear el pickle.
pickle.dump(trials, trials_pickle)

# Finalmente hay que cerrar la instancia del pickle.
trials_pickle.close()

### Lectura de Trials

_Observación_: Recordar que luego hay que aumentar el número de `max_evals`, en `fmin`, para seguir acumulando pruebas.

Si se hicieron 60 pruebas entre los Trials anteriores y se quieren hacer 20 más entonces hay que usar `max_evals=80`.

In [ ]:
# String con dirección de almacenamiento y nombre del archivo .pkl.
file_name = 'pickle/Trials_LGBMClassifier05.pkl'

# Se crea una variable para el archivo pickle y se abre con la dirección del
# archivo en modo lectura. 
trials_pkl = open(file_name, 'rb')

# Se carga el modelo pasando la variable creada, en la misma se encuentra
# la dirección del archivo pickle.
trials = pickle.load(trials_pkl)
print('Modelo cargado')